<a href="https://colab.research.google.com/github/agorastats/mitosisDEEP/blob/master/colabs/dataGeneratorTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages
import os
import tensorflow as tf
import numpy as np 
import random
import cv2
from tensorflow import keras

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
# get data from google drive (unzip images to read it)
!unzip 'drive/MyDrive/mitosis_data/dataset_20220207.zip' -d "/content/data"/

In [ ]:
# import mitosisDEEP github project
!rm -rf mitosisDEEP  # remove if exists
# Clone the entire repo
!git clone -l -s https://github.com/agorastats/mitosisDEEP.git mitosisDEEP
import sys
sys.path.insert(0, 'mitosisDEEP/')

Cloning into 'mitosisDEEP'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (370/370), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 370 (delta 155), reused 277 (delta 74), pack-reused 0
Receiving objects: 100% (370/370), 113.10 MiB | 40.66 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
# define generator
!pip install opencv-python.headless~=4.1.2.30
!pip install albumentations~=1.1.0

from mitosisDEEP.dataGeneratorProcess import DataGenerator
import pandas as pd 
df=pd.read_csv('data/infoDF.csv', sep=';')
gen = DataGenerator(df=df, img_path='data/images', mask_path='data/masks', shuffle=False)

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1828: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1802: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


FileNotFoundError: ignored

In [ ]:
df.head(3)

In [ ]:
img, masks = gen.__getitem__(0)

In [ ]:
# keras utils
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
  denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
  return 1 - numerator / denominator

In [ ]:
# load pretrained unet model
from keras.models import load_model
model = load_model("drive/MyDrive/mitosis_data/UNET-Best.h5", compile=False)
# model.save_weights("drive/MyDrive/mitosis_data/pretrained_weights.h5")

# build model
# adapted Adam optimizer with low learning rate
opt_adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt_adam, loss=dice_loss, metrics=[dice_coef])
# model.load_weights("drive/MyDrive/mitosis_data/pretrained_weights.h5")
# model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet.h5")
model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet_v2.h5")

In [ ]:
# training step

# early stop if patience epochs not improve val_loss
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# reduce learning rate by factor if not improve val_loss in patience epochs
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001, verbose=1)

# path to save new weights, only save best ones
output_name = 'proves_dataset20220207_pretrained_unet_v3'
# output_weights_path="drive/MyDrive/mitosis_data/%s-{epoch:02d}-{val_dice_coef:.3f}.h5" % str(output_name)
output_weights_path="drive/MyDrive/mitosis_data/%s.h5" % str(output_name)
checkpoint = tf.keras.callbacks.ModelCheckpoint(output_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)


# callback save logs as csv
from keras.callbacks import CSVLogger
from mitosisDEEP.utils.loadAndSaveResults import store_data_frame
store_data_frame(pd.DataFrame(), 'logsCSV/%s.csv' % str(output_name))
csv_logger = CSVLogger('logsCSV/%s.csv' % str(output_name), append=True, separator=';')


import shutil
class UpdateLoggerToDrive(tf.keras.callbacks.Callback):
    def __init__(self, N, output_name):
        self.N = N
        self.epoch = 0
        self.output_name = output_name
        self.logCSV = 'logsCSV/%s.csv' % str(self.output_name)

    def on_batch_end(self, epoch, logs={}):
        if self.epoch % self.N == 0:
          shutil.copy(self.logCSV, '/content/drive/MyDrive/mitosis_data/logs/%s.csv' % str(self.output_name))
        self.epoch += 1

update_log_to_drive = UpdateLoggerToDrive(5, output_name)


# tensorboard to visualize model
# ref1: https://colab.research.google.com/github/Yash0330/Callbacks-in-Keras/blob/master/Introduction_to_callbacks.ipynb
# ref2: https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb#scrollTo=lpUO9HqUKP6z

# !rm -rf ./logs/ # to delete all previous runs
# import datetime
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0, write_graph=True, write_images=True)  # add tensorboard to callbacks
# %tensorboard --logdir logs



# random choice for val images
idx = np.random.choice(len(df), int(len(df) * 0.7), replace=False)
trainDF = df.iloc[idx, :]
valDF = df.iloc[~idx, :]
training_generator = DataGenerator(df=trainDF, batch_size=4, img_path='data/images', mask_path='data/masks', shuffle=True)
validation_generator = DataGenerator(df=valDF, batch_size=4,  img_path='data/images', mask_path='data/masks', augmentations=None) 
EPOCH_NUMBER=100



history = model.fit(training_generator, validation_data=validation_generator, verbose=1,
                    epochs=EPOCH_NUMBER, callbacks=[early_stop, checkpoint, reduce_lr, csv_logger, update_log_to_drive]) # add tensorbard callback

# get_callbacks_list(output_name)



Epoch 1/100
657/657 [==============================] - ETA: 0s - loss: 0.5704 - dice_coef: 0.6222
Epoch 00001: val_loss improved from inf to 0.59543, saving model to drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet_v3.h5
657/657 [==============================] - 124s 181ms/step - loss: 0.5704 - dice_coef: 0.6222 - val_loss: 0.5954 - val_dice_coef: 0.5665 - lr: 0.0010
Epoch 2/100
657/657 [==============================] - ETA: 0s - loss: 0.5790 - dice_coef: 0.5997
Epoch 00002: val_loss did not improve from 0.59543
657/657 [==============================] - 118s 179ms/step - loss: 0.5790 - dice_coef: 0.5997 - val_loss: 0.7515 - val_dice_coef: 0.4574 - lr: 0.0010
Epoch 3/100
657/657 [==============================] - ETA: 0s - loss: 0.5847 - dice_coef: 0.5877
Epoch 00003: val_loss improved from 0.59543 to 0.55531, saving model to drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet_v3.h5
657/657 [==============================] - 116s 177ms/step - loss: 0.58

In [ ]:
# evaluate mitosis
# ref: https://www.youtube.com/watch?v=HrGn4uFrMOM
from google.colab.patches import cv2_imshow
!pip install patchify
from patchify import patchify, unpatchify
PATCH_SIZE  = 256
# test_path = 'drive/MyDrive/mitosis_data/A00_01.bmp'
# test_path = 'drive/MyDrive/mitosis_data/A01_09.bmp'  # validation
# test_path = 'drive/MyDrive/mitosis_data/30.jpg'   
# test_path = 'drive/MyDrive/mitosis_data/23.jpg'   
test_path = 'drive/MyDrive/mitosis_data/25.jpg'   
test_image = cv2.imread(test_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
# cv2_imshow(test_image)

In [ ]:
# predict using patchify
size_x = (test_image.shape[1]//PATCH_SIZE)*PATCH_SIZE   # nearest size divisible by our patch size
size_y = (test_image.shape[0]//PATCH_SIZE)*PATCH_SIZE   # nearest size divisible by our patch size
print('test image size: ', test_image.shape)
test_image = cv2.resize(test_image, (size_x, size_y))
print('test image resized size: ', test_image.shape)

patches = patchify(test_image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_SIZE)  # step same as patch for not overlap patches
patches = patches[:, :, 0, :, :, :]
predicted_patches = []
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        single_patch = patches[i,j,:,:, :] / 255.   # normalize
        single_patch = np.expand_dims(single_patch, axis=0) # (256,256,3) to (1,256,256,3)
        single_patch_prediction = (model.predict(single_patch) > 0.5).astype(np.uint8)
        predicted_patches.append(single_patch_prediction[0, :, :])


predicted_patches_reshaped = np.reshape(predicted_patches, [patches.shape[0], patches.shape[1], patches.shape[2], patches.shape[3]]) 
reconstructed_image = unpatchify(predicted_patches_reshaped, (test_image.shape[0], test_image.shape[1]))
cv2_imshow(reconstructed_image * 255.) # multiply to visualize graph


In [ ]:
# !pip install patchify
from mitosisDEEP.evaluateLargeImageProcess import EvaluateLargeImageProcess

evalImg = EvaluateLargeImageProcess(df=testDF, img_path='/content/drive/MyDrive/mitosis_data', model=model, output_info='output')

In [ ]:
# test_path = 'drive/MyDrive/mitosis_data/A00_01.bmp'
# test_path = 'drive/MyDrive/mitosis_data/A01_09.bmp'  # validation
# test_path = 'drive/MyDrive/mitosis_data/30.jpg'   
# test_path = 'drive/MyDrive/mitosis_data/23.jpg' 

testDF= pd.DataFrame({'id': ['A00_01.bmp','A01_09.bmp', '30.jpg', '23.jpg']})
testDF

,id
0,A00_01.bmp
1,A01_09.bmp
2,30.jpg
3,23.jpg


In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)
evalImg.run(dict())

INFO:root:__predict image: A00_01.bmp
INFO:root:test image size: (2084, 2084)
INFO:root:test image resized size: (2048, 2048)
INFO:root:__predict patches
INFO:root:__reconstruct image with patches
INFO:root:__predict image: A01_09.bmp
INFO:root:test image size: (2084, 2084)
INFO:root:test image resized size: (2048, 2048)
INFO:root:__predict patches
INFO:root:__reconstruct image with patches
INFO:root:__predict image: 30.jpg
INFO:root:test image size: (1377, 2560)
INFO:root:test image resized size: (1280, 2560)
INFO:root:__predict patches
INFO:root:__reconstruct image with patches
INFO:root:__predict image: 23.jpg
INFO:root:test image size: (1173, 2560)
INFO:root:test image resized size: (1024, 2560)
INFO:root:__predict patches
INFO:root:__reconstruct image with patches
